# ***ZINDI COMPETITION: BIOMASS PREDICTION***
Notebook done by : Team **TUNIG**


## Importations and Data Download

In [1]:
import h5py
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
import lightgbm
!pip install catboost
import catboost
from sklearn.model_selection import KFold
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 6.0 MB/s eta 0:00:00


In [2]:
!wget -q  https://share.phys.ethz.ch/~pf/albecker/abc/09072022_1154_train.h5
!wget -q  https://share.phys.ethz.ch/~pf/albecker/abc/09072022_1154_val.h5
!wget -q https://share.phys.ethz.ch/~pf/albecker/abc/09072022_1154_test.h5

## Load datasets

In [3]:
trainset = h5py.File("09072022_1154_train.h5", "r")
validateset = h5py.File("09072022_1154_val.h5", "r")
testset = h5py.File("09072022_1154_test.h5", "r")

In [4]:
# attributes of trainset
trainset.keys()

<KeysViewHDF5 ['agbd', 'cloud', 'images', 'lat', 'lon', 'scl']>

In [5]:
# train
train_images = np.array(trainset['images'],dtype=np.float64)
train_images = train_images.transpose(0,3,1,2)
train_cloud = np.array(trainset['cloud'],dtype=np.float64)
train_cloud = train_cloud.transpose(0,3,1,2)
train_lat = np.array(trainset['lat'],dtype=np.float64)
train_lat = train_lat.transpose(0,3,1,2)
train_lon = np.array(trainset['lon'],dtype=np.float64)
train_lon = train_lon.transpose(0,3,1,2)
train_scl = np.array(trainset['scl'],dtype=np.float64)
train_scl = train_scl.transpose(0,3,1,2)
train_biomasses = np.array(trainset['agbd'],dtype=np.float64)

In [6]:
# validate
validate_images = np.array(validateset['images'],dtype=np.float64)
validate_images = validate_images.transpose(0,3,1,2)
validate_cloud = np.array(validateset['cloud'],dtype=np.float64)
validate_cloud = validate_cloud.transpose(0,3,1,2)
validate_lat = np.array(validateset['lat'],dtype=np.float64)
validate_lat = validate_lat.transpose(0,3,1,2)
validate_lon = np.array(validateset['lon'],dtype=np.float64)
validate_lon = validate_lon.transpose(0,3,1,2)
validate_scl = np.array(validateset['scl'],dtype=np.float64)
validate_scl = validate_scl.transpose(0,3,1,2)
validate_biomasses = np.array(validateset['agbd'],dtype=np.float64)


In [7]:
# test 
test_images = np.array(testset['images'],dtype=np.float32)
test_images = test_images.transpose(0,3,1,2)
test_cloud = np.array(testset['cloud'],dtype=np.float64)
test_cloud = test_cloud.transpose(0,3,1,2)
test_lat = np.array(testset['lat'],dtype=np.float64)
test_lat = test_lat.transpose(0,3,1,2)
test_lon = np.array(testset['lon'],dtype=np.float64)
test_lon = test_lon.transpose(0,3,1,2)
test_scl = np.array(testset['scl'],dtype=np.float64)
test_scl = test_scl.transpose(0,3,1,2)
test_biomasses = np.array(testset['agbd'],dtype=np.float32)

In [8]:
print(f"train dataset size {train_images.shape} train lab size {train_biomasses.shape}")
print()
print(f"validate dataset size {validate_images.shape} validate lab size {validate_biomasses.shape}")
print()
print(f"test dataset size {test_images.shape} test lab size {test_biomasses.shape}")

train dataset size (25036, 12, 15, 15) train lab size (25036,)

validate dataset size (5174, 12, 15, 15) validate lab size (5174,)

test dataset size (5190, 12, 15, 15) test lab size (5190,)


## Data Cleaning and Feature Extraction

One of the most things that helped us to reduce overfitting of unecessary  features is to remove all the samples that have biomass lower than 45


In [9]:
train_biomasses1=train_biomasses[(train_biomasses>=45.0)]
train_img=train_images[(train_biomasses>=45.0)]

validate_biomasses1=validate_biomasses[(validate_biomasses>=45.0)]
validate_images=validate_images[(validate_biomasses>=45.0)]

test_biomasses1=test_biomasses[(test_biomasses>=45.0)]
test_images=test_images[(test_biomasses>=45.0)]

Also we created a function that will extract beneficial features based on the different bands that we have as example we will get the vegetation index and several bands combination and also we applied pca and selected the first 5 principal components.

In [10]:
import numpy as np
import skimage.feature
import sklearn.decomposition

def extract_features(X, cloud, scl, lat, lon):
    num_samples, num_channels, height, width = X.shape
    X = X.reshape(num_samples, num_channels, height * width)

    # Calculate vegetation indices
    S2REP = 705 + 35 * ((((X[:, 6, :] + X[:, 3, :])/2) - X[:, 4, :])/(X[:, 5, :] - X[:, 4, :])) 
    CCCI = ((X[:, 7, :] - X[:, 4, :]) / (X[:, 7, :] + X[:, 4, :])) / ((X[:, 7, :] - X[:, 3, :]) / (X[:, 7, :] + X[:, 3, :])) 
    MCARI = ((X[:, 4, :] - X[:, 3, :]) - 2 * (X[:, 4, :] - X[:, 2, :])) * (X[:, 4, :] / X[:, 3, :])  
    TCARI = 3 * ((X[:, 4, :] - X[:, 3, :]) - 0.2 * (X[:, 4, :] - X[:, 2, :]) * (X[:, 4, :] / X[:, 3, :])) 
    PVI = (X[:, 7, :] - 0.3 * X[:, 3, :] - 0.5) / ((1 + 0.3 * 2) ** (1/2.0)) 
    ndvi = (X[:, 7, :] - X[:, 3, :]) / (X[:, 7, :] + X[:, 3, :])
    evi = 2.5 * (X[:, 7, :] - X[:, 3, :]) / (X[:, 7, :] + 6 * X[:, 3, :] - 7.5 * X[:, 1, :] + 1)
    savi = (X[:, 7, :] - X[:, 3, :]) / (X[:, 7, :] + X[:, 3, :] + 0.5)
    mndwi = (X[:, 2, :] - X[:, 7, :]) / (X[:, 2, :] + X[:, 7, :])



    ARVI =  (X[:, 7, :] - (2 * X[:, 3, :]) + X[:, 1, :]) / (X[:, 7, :] + (2 * X[:, 3, :]) + X[:, 1, :])
    SIPI = (X[:, 7, :] - X[:, 1, :]) / (X[:, 7, :] - X[:, 3, :])
    RENDVI = (X[:, 5, :] - X[:, 4, :]) / (X[:, 5, :] + X[:, 4, :]) 
    MRESR = (X[:, 5, :] - X[:, 0, :]) / (X[:, 4, :] - X[:, 0, :])

    # CANOLA
    RYI = X[:, 2, :] / X[:, 1, :]
    NDYI = (X[:, 2, :] - X[:, 1, :]) / (X[:, 2, :] + X[:, 1, :])
    DYI = X[:, 2, :] - X[:, 1, :]
    ACI = X[:, 7, :] * (X[:, 3, :] + X[:, 2, :])

    # WEED
    CVI = (X[:, 7, :] / X[:, 2, :]) * (X[:, 3, :] / X[:, 2, :])
    AVI = (X[:, 7, :] * (1 - X[:, 3, :]) * (X[:, 7, :] - X[:, 3, :]))
    SI= ((1 - X[:, 1, :]) * (1 - X[:, 2, :]) * (1 - X[:, 3, :]))
    BSI= ((X[:, 10, :] + X[:, 3, :]) - (X[:, 7, :] + X[:, 1, :])) / ((X[:, 10, :] + X[:, 3, :]) + (X[:, 7, :] + X[:, 1, :]))

    # WINE GRAPES

    MTCI = (X[:, 5, :] - X[:, 4, :])/(X[:, 4, :] - X[:, 3, :])
    NPCRI = (X[:, 3, :] - X[:, 1, :]) / (X[:, 3, :] + X[:, 1, :])

   

    # ROOIBOS
    BAI = 1/((0.1 - X[:, 3, :]) ** 2 + (0.06 - X[:, 7, :]) ** 2)
    MTVI2 = list(1.5*(1.2 * (i - j) - 2.5 * (k - j))* ((2 * i + 1)**2-(6 * i - 5 * k ** (1/2.0)) - 0.5)**(1/2.0) for i, j, k in zip(X[:, 7, :], X[:, 2, :], X[:, 3, :]))
    MTVI2=np.array(MTVI2)
    NDSI = (X[:, 2, :] - X[:, 10, :]) / (X[:, 2, :] + X[:, 10, :])



    # DRYNESS / DROUGHT
    NDMI = (X[:, 7, :] - X[:, 10, :])/(X[:, 7, :] + X[:, 10, :]) 
    TNDVI = [(x)**(1/2.0) for x in ((X[:, 7, :] - X[:, 3, :]) / (X[:, 7, :] + X[:, 3, :]) + 0.5)]
    TNDVI=np.array(TNDVI)


    # GENERAL
    TVI = (120 * (X[:, 5, :] - X[:, 2, :]) - 200 * (X[:, 3, :] - X[:, 2, :])) / 2
    EXG = 2 * X[:, 2, :] - X[:, 3, :] - X[:, 1, :]
    PSRI = (X[:, 3, :] - X[:, 1, :]) / X[:, 5, :]

 
    
    #Apply PCA
    pca = sklearn.decomposition.PCA(n_components=5)
    X_pca = pca.fit_transform(X.reshape(num_samples, -1))
    X_pca.reshape(num_samples, -1),
    print(X_pca.shape)
    
   
    # Combine features
    features = np.concatenate([ndvi.reshape(num_samples, -1),
                               evi.reshape(num_samples, -1),PSRI.reshape(num_samples, -1),EXG.reshape(num_samples, -1)
                               ,TVI.reshape(num_samples, -1),
                               savi.reshape(num_samples, -1),TNDVI.reshape(num_samples, -1)
                               ,NDMI.reshape(num_samples, -1),
                               mndwi.reshape(num_samples, -1),
                               PVI.reshape(num_samples, -1)
                               ,
                               TCARI.reshape(num_samples, -1),NPCRI.reshape(num_samples, -1),
                               MTCI.reshape(num_samples, -1),
                               MCARI.reshape(num_samples, -1),BSI.reshape(num_samples, -1),SI.reshape(num_samples, -1),
                               AVI.reshape(num_samples, -1),CVI.reshape(num_samples, -1),
                               CCCI.reshape(num_samples, -1),ACI.reshape(num_samples, -1),DYI.reshape(num_samples, -1)
                               ,NDYI.reshape(num_samples, -1),RYI.reshape(num_samples, -1),
                               S2REP.reshape(num_samples, -1),MRESR.reshape(num_samples, -1),RENDVI.reshape(num_samples, -1),SIPI.reshape(num_samples, -1),
                                ARVI.reshape(num_samples, -1),
                               X_pca,
                
                               ], axis=1)
    
    return features


In [11]:
#perform extracted features for all the training data that we have
tr=extract_features(train_img,train_cloud,train_scl,train_lat,train_lon)
vl=extract_features(validate_images,validate_cloud,validate_scl,validate_lat,validate_lon)
te=extract_features(test_images,test_cloud,test_scl,test_lat,test_lon)

(7280, 5)
(1491, 5)
(1601, 5)


In [12]:
#concatinate all of them to get one training data with extracted features
X=np.concatenate([tr,vl,te],axis=0)
y=np.concatenate([train_biomasses1,validate_biomasses1,test_biomasses1],axis=0)

## Test Reading and Preprocessing

Now we read the test data and extract their features as well same to what we did to train data

**NOTE** : we are assuming that all the test files were uploaded to colab or it will result in an error of data existence

In [13]:
testset = h5py.File("/content/images_test.h5", "r")
test_lat=h5py.File("/content/lat_test.h5", "r")
test_scl=h5py.File("/content/scl_test.h5", "r")
test_cloud=h5py.File("/content/cloud_test.h5", "r")
test_lon=h5py.File("/content/lon_test.h5", "r")

In [14]:
test_images = np.array(testset['images'],dtype=np.float32)
test_images = test_images.transpose(0,3,1,2)
test_cloud = np.array(test_cloud['cloud'],dtype=np.float64)
test_cloud = test_cloud.transpose(0,3,1,2)
test_lat = np.array(test_lat['lat'],dtype=np.float64)
test_lat = test_lat.transpose(0,3,1,2)
test_lon = np.array(test_lon['lon'],dtype=np.float64)
test_lon = test_lon.transpose(0,3,1,2)
test_scl = np.array(test_scl['scl'],dtype=np.float64)
test_scl = test_scl.transpose(0,3,1,2)

In [15]:
X_test=extract_features(test_images,test_cloud,test_scl,test_lat,test_lon)

(90, 5)


## Modeling

For modeling we used a 5 folds cross validation and performed the regression using lightgbm regressor. We also did the inference in the same loop after that we we will take the mean of the predictions

In [16]:
kf=KFold(n_splits=5,shuffle=True,random_state=2022)

val_score_k=[]
train_score_k=[]
test_pred=[]
train_score=[]
val_score=[]


for tr_index,vl_index in kf.split(X) :
  X_train,X_val=X[tr_index],X[vl_index]
  y_train,y_val=y[tr_index],y[vl_index]
  model=lightgbm.LGBMRegressor(max_depth=8,random_state=2022)
  model.fit(X_train,y_train,eval_set=[(X_val,y_val)],early_stopping_rounds=20)
  val_score.append(np.sqrt(mean_squared_error(y_val, model.predict(X_val))))
  train_score.append(np.sqrt(mean_squared_error(y_train, model.predict(X_train))))
  test_pred.append(model.predict(X_test))

[1]	valid_0's l2: 6462.08
[2]	valid_0's l2: 6440.74
[3]	valid_0's l2: 6418.97
[4]	valid_0's l2: 6387.28
[5]	valid_0's l2: 6369.7
[6]	valid_0's l2: 6347.09
[7]	valid_0's l2: 6330.21
[8]	valid_0's l2: 6323.42
[9]	valid_0's l2: 6305.89
[10]	valid_0's l2: 6282.03
[11]	valid_0's l2: 6266.67
[12]	valid_0's l2: 6265.83
[13]	valid_0's l2: 6257.86
[14]	valid_0's l2: 6229.64
[15]	valid_0's l2: 6228.31
[16]	valid_0's l2: 6225.41
[17]	valid_0's l2: 6230.13
[18]	valid_0's l2: 6219.23
[19]	valid_0's l2: 6204.91
[20]	valid_0's l2: 6178.21
[21]	valid_0's l2: 6167.39
[22]	valid_0's l2: 6167.83
[23]	valid_0's l2: 6162.49
[24]	valid_0's l2: 6153.05
[25]	valid_0's l2: 6147.37
[26]	valid_0's l2: 6131.3
[27]	valid_0's l2: 6131.56
[28]	valid_0's l2: 6123.49
[29]	valid_0's l2: 6117.56
[30]	valid_0's l2: 6119.76
[31]	valid_0's l2: 6114.75
[32]	valid_0's l2: 6116.7
[33]	valid_0's l2: 6122.74
[34]	valid_0's l2: 6119.89
[35]	valid_0's l2: 6128.38
[36]	valid_0's l2: 6135.16
[37]	valid_0's l2: 6137.45
[38]	valid_0'

In [17]:
final_predictions=np.mean(test_pred,axis=0)
final_predictions

array([ 89.0011386 , 102.16344842,  92.68908486,  99.43787087,
        92.41049998, 104.98018867,  94.76564687,  98.950172  ,
       118.93034709,  89.6654156 , 104.03271668, 106.60514263,
       133.36633581, 109.40366881, 137.62553942, 120.91451384,
        88.52398539, 126.76210977,  98.49170995,  99.09379901,
       114.29272927, 115.80509329, 108.15152848, 110.66366776,
       100.0919276 , 116.58496549,  97.53251341,  93.56968777,
       106.69319188, 125.02438487, 123.57778723,  96.35349885,
       104.5147828 , 123.99043519,  93.34273117,  91.93001701,
       107.69224341, 136.1630684 , 107.38968669,  97.98006682,
       130.57527274, 120.64382877, 114.87151772, 101.4388298 ,
        97.33355589,  88.48896424,  88.41071475, 108.25225257,
       108.81310701, 122.96753101, 111.77842362, 100.30075866,
       106.9455734 , 126.38347375, 106.13602364, 126.81480136,
        95.5307752 ,  98.68007833,  90.12540062, 107.79165933,
       103.51339103,  99.02007489, 109.87062009,  97.99

In [18]:
print('Train score in Cross Validation is :',np.mean(train_score))

Train score in Cross Validation is : 60.25991656267164


In [19]:
print('Validation score in Cross Validation is :',np.mean(val_score))

Validation score in Cross Validation is : 78.37875641917644


## Make the predictions into a submission file

In [20]:
ID_S2_pair = pd.read_csv('/content/UniqueID-SentinelPair.csv')

preds = pd.DataFrame({'Target':final_predictions}).rename_axis('S2_idx').reset_index()
preds = ID_S2_pair.merge(preds, on='S2_idx').drop(columns=['S2_idx'])

In [21]:
preds.to_csv('third_place_TUNIG.csv', index=False)

In [22]:
preds

,ID,Target
0,ID_1EB0DGFP07,97.333556
1,ID_844T2PSXTK,90.125401
2,ID_4MCV3S8MLN,93.569688
3,ID_L7441JV5F3,125.783587
4,ID_5GUVM4YEWZ,98.680078
...,...,...
85,ID_MEW6189J1B,120.643829
86,ID_TH9HRUXGTP,106.945573
87,ID_GPC7YS3JG8,136.400287
88,ID_1P7PJMPV0R,89.001139
